## Preprocessing and Training Data Development - Vacancy Rates 

**Note: THIS NOTEBOOK WAS NOT COMPLETED BECAUSE THE .FIT() FUNCTION TOOK FAR TOO LONG AND WE DID NOT GET ANY RESULTS**


Goal:  Create a cleaned development dataset you can use to complete the modeling step of your project


#### Steps: 
● 1. Create dummy or indicator features for categorical variables

● 2. Standardize the magnitude of numeric features using a scaler

● 3. Split into testing and training datasets

In [2]:
#imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime
from pandas_profiling import ProfileReport

In [3]:
#load data
path= '/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/interim'
os.chdir(path) 

In [4]:
# load cleaned data
df = pd.read_csv('master_complete_for_EDA.csv', dtype={'Zipcode': object})
df

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice,Vacancy_Rate%,MOE-VacancyRate%,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
0,02333,1368.536,2011,8782.0,MA,East Bridgewater,Boston-Cambridge-Newton,Plymouth County,NaN,3.024027,2.199925,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
1,02338,1311.076,2011,11179.0,MA,Halifax,Boston-Cambridge-Newton,Plymouth County,274920.17,3.116343,2.948791,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
2,02339,1484.626,2011,8621.0,MA,Hanover,Boston-Cambridge-Newton,Plymouth County,415097.50,4.464646,2.066438,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
3,02341,1266.816,2011,10079.0,MA,Hanson,Boston-Cambridge-Newton,Plymouth County,NaN,3.586322,2.340722,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
4,02343,1524.006,2011,9640.0,MA,Holbrook,Boston-Cambridge-Newton,Norfolk County,247510.42,3.732901,2.926524,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264955,98279,1059.870,2018,23400.0,WA,Olga,NaN,San Juan County,552805.42,51.219512,10.993457,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264956,98280,993.850,2018,25265.0,WA,Eastsound,NaN,San Juan County,678499.00,51.329243,12.777549,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264957,98311,1533.500,2018,4981.0,WA,Bremerton,Bremerton-Silverdale,Kitsap County,314320.83,6.540162,1.960476,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264958,98326,778.990,2018,26185.0,WA,Clallam Bay,Port Angeles,Clallam County,150193.17,28.537736,14.679524,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000


In [5]:
#drop margin of error of vacancy rate
df.drop('MOE-VacancyRate%', axis=1, inplace=True)

In [6]:
df.dtypes

Zipcode                   object
RentPrice                float64
Year                       int64
SizeRank                 float64
State                     object
City                      object
Metro                     object
CountyName                object
HomePrice                float64
Vacancy_Rate%            float64
int_rate                 float64
med_hIncome              float64
uspop_growth             float64
unemplt_rate             float64
newHouse_starts          float64
resConstruct_spending    float64
dtype: object

In [7]:
#not needed for this dataframe
'''
#Create a new dataframe, setting the index to 'Year'
df = DF.set_index('Year')
#Save the DATE labels 
df_index = df.index
#Save the column names
df_columns = df.columns
df.head()
'''

"\n#Create a new dataframe, setting the index to 'Year'\ndf = DF.set_index('Year')\n#Save the DATE labels \ndf_index = df.index\n#Save the column names\ndf_columns = df.columns\ndf.head()\n"

In [8]:
#not needed for this dataframe
'''
#split into two dataframes for future modeling and predicting vacancy rates in 2019-2020
df = df[df.Year < 2019]
df_2019_2020 = df[df.Year > 2018]
'''

'\n#split into two dataframes for future modeling and predicting vacancy rates in 2019-2020\ndf = df[df.Year < 2019]\ndf_2019_2020 = df[df.Year > 2018]\n'

In [9]:
#check NaNs
df.isna().sum()

Zipcode                      0
RentPrice                19907
Year                         0
SizeRank                 27056
State                    27056
City                     27056
Metro                    83008
CountyName               27056
HomePrice                37378
Vacancy_Rate%                0
int_rate                     0
med_hIncome                  0
uspop_growth                 0
unemplt_rate                 0
newHouse_starts              0
resConstruct_spending        0
dtype: int64

In [10]:
#drop NaNs
df.dropna(inplace=True)
df

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice,Vacancy_Rate%,int_rate,med_hIncome,uspop_growth,unemplt_rate,newHouse_starts,resConstruct_spending
1,02338,1311.076,2011,11179.0,MA,Halifax,Boston-Cambridge-Newton,Plymouth County,274920.17,3.116343,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
2,02339,1484.626,2011,8621.0,MA,Hanover,Boston-Cambridge-Newton,Plymouth County,415097.50,4.464646,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
4,02343,1524.006,2011,9640.0,MA,Holbrook,Boston-Cambridge-Newton,Norfolk County,247510.42,3.732901,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
5,02346,1310.016,2011,5289.0,MA,Middleborough,Boston-Cambridge-Newton,Plymouth County,264492.50,7.960256,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
6,02347,1307.736,2011,9579.0,MA,Lakeville,Boston-Cambridge-Newton,Plymouth County,309743.67,11.565968,0.750000,57021.0,0.720018,8.933333,611.916667,255208.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264950,98134,1909.580,2018,28159.0,WA,Seattle,Seattle-Tacoma-Bellevue,King County,438970.00,13.580247,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264953,98233,1413.890,2018,7640.0,WA,Burlington,Mount Vernon-Anacortes,Skagit County,317426.75,4.853765,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264957,98311,1533.500,2018,4981.0,WA,Bremerton,Bremerton-Silverdale,Kitsap County,314320.83,6.540162,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000
264958,98326,778.990,2018,26185.0,WA,Clallam Bay,Port Angeles,Clallam County,150193.17,28.537736,2.458333,64324.0,0.522337,3.891667,1248.250000,564448.750000


In [11]:
#check unique values for each column
df['CountyName'].value_counts()/len(df)*100

Los Angeles County    1.287081
Jefferson County      1.196193
Washington County     1.155147
Montgomery County     0.985687
Orange County         0.944055
                        ...   
Trousdale County      0.004691
Long County           0.004691
Stanley County        0.004105
Lynn County           0.003518
Eddy County           0.001759
Name: CountyName, Length: 1217, dtype: float64

In [12]:
#check partition sizes with a 80/20 train/test split
print('train size:', len(df) * .8, 'test size:', len(df) * .2)

train size: 136432.80000000002 test size: 34108.200000000004


###  1. Create dummy or indicator features for categorical variables
Hint: you’ll need to think about your old favorite pandas functions here like
get_dummies() . Consult this guide for help.
<https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40>

In [13]:
#change zipcode so it's not turned into a dummy variable
df.Zipcode = df.Zipcode.astype('int')
df.dtypes

Zipcode                    int64
RentPrice                float64
Year                       int64
SizeRank                 float64
State                     object
City                      object
Metro                     object
CountyName                object
HomePrice                float64
Vacancy_Rate%            float64
int_rate                 float64
med_hIncome              float64
uspop_growth             float64
unemplt_rate             float64
newHouse_starts          float64
resConstruct_spending    float64
dtype: object

In [14]:
#get dummy variables for 'object' columns 
df = pd.get_dummies(df)
df

,Zipcode,RentPrice,Year,SizeRank,HomePrice,Vacancy_Rate%,int_rate,med_hIncome,uspop_growth,unemplt_rate,...,CountyName_Yates County,CountyName_Yavapai County,CountyName_Yazoo County,CountyName_Yell County,CountyName_Yellowstone County,CountyName_Yolo County,CountyName_York County,CountyName_Yuba County,CountyName_Yuma County,CountyName_Zapata County
1,2338,1311.076,2011,11179.0,274920.17,3.116343,0.750000,57021.0,0.720018,8.933333,...,0,0,0,0,0,0,0,0,0,0
2,2339,1484.626,2011,8621.0,415097.50,4.464646,0.750000,57021.0,0.720018,8.933333,...,0,0,0,0,0,0,0,0,0,0
4,2343,1524.006,2011,9640.0,247510.42,3.732901,0.750000,57021.0,0.720018,8.933333,...,0,0,0,0,0,0,0,0,0,0
5,2346,1310.016,2011,5289.0,264492.50,7.960256,0.750000,57021.0,0.720018,8.933333,...,0,0,0,0,0,0,0,0,0,0
6,2347,1307.736,2011,9579.0,309743.67,11.565968,0.750000,57021.0,0.720018,8.933333,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264950,98134,1909.580,2018,28159.0,438970.00,13.580247,2.458333,64324.0,0.522337,3.891667,...,0,0,0,0,0,0,0,0,0,0
264953,98233,1413.890,2018,7640.0,317426.75,4.853765,2.458333,64324.0,0.522337,3.891667,...,0,0,0,0,0,0,0,0,0,0
264957,98311,1533.500,2018,4981.0,314320.83,6.540162,2.458333,64324.0,0.522337,3.891667,...,0,0,0,0,0,0,0,0,0,0
264958,98326,778.990,2018,26185.0,150193.17,28.537736,2.458333,64324.0,0.522337,3.891667,...,0,0,0,0,0,0,0,0,0,0


### 2. Split into testing and training datasets
Hint: don’t forget your sklearn functions here, like train_test_split().

In [15]:
#define variable X, y
X = df.drop('Vacancy_Rate%', axis=1)
y = df['Vacancy_Rate%']

In [16]:
#train test split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [17]:
#train, test split for timeseries
tss = TimeSeriesSplit(n_splits = 5)
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

### Establish Baseline Measurement Comparisons
Using a Dummy Regressor see what R2, MSE, and MAE would be if the mean of the DataFrames were used

In [18]:
#initial not even a model
train_mean = y_train.mean()

print(train_mean)

13.87446172393452


In [19]:
#Fit the dummy regressor on the training data
dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
#create dummy regressor predictions 
y_tr_pred = dumb_reg.predict(X_train)
#Make prediction with the single value of the (training) mean.
y_te_pred = train_mean * np.ones(len(y_test))
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(0.0, -0.002354908363507935)

In [20]:
#establish baseline for mean absolute error and mean square error 
print('MAEs:', mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred))
print('MSEs:', mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred))

MAEs: 8.381084526531904 8.866017693533786
MSEs: 153.09577760438876 171.1263648405277


### 3. Standardize the magnitude of numeric features using a scaler
Hint: you might need to employ Python code like this:

In [21]:
#skipping this step for now, see if we don't need it. 
#also dummy variables are already input, so may need to move this step to before that.
'''
scaler = StandardScaler()
#fit the scaler on the training set
scaler.fit(X_train)
#apply the scaling to both the train and test split
X_tr_scaled = scaler.transform(X_train)
X_te_scaled = scaler.transform(X_test)
'''

'\nscaler = StandardScaler()\n#fit the scaler on the training set\nscaler.fit(X_train)\n#apply the scaling to both the train and test split\nX_tr_scaled = scaler.transform(X_train)\nX_te_scaled = scaler.transform(X_test)\n'

#### Initial Model: Train the model on the train split

In [36]:
from time import time
rows = 10000
t = time()

lm = LinearRegression().fit(X_train.head(rows), y_train.head(rows))
print('model fitting for Linear regression, on {} rows. Took {} seconds'.format(rows, time()-t))

model fitting for Linear regression, on 10000 rows. Took 711.7782809734344 seconds


In [37]:
#Make predictions using the model on both train and test splits
y_tr_pred = lm.predict(X_train)
y_te_pred = lm.predict(X_test)

In [38]:
#Assess model performance
# r^2 - train, test
r2 = r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)
print('r2:', r2)

r2: (-1.9485290695152772e+22, -6.32484890277947e+22)


**This is markedly better performance than when using Dummy variable/mean for R^2 (see earlier):**

Dummy R2 = (0.0, -0.001031839268772039)

In [39]:
#MAE - train, test
mae = mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)
print('mae:', mae)

mae: (1384284617313.662, 3284973530594.9917)


In [40]:
# MSE - train, test
mse = mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)
print('mse:', mse)

mse: (2.9831157308219744e+24, 1.0798055577593212e+25)


**This is markedly better performance than when using Dummy variable/mean for R^2 (see earlier):**

Dummy -

MAEs: 5.14329246126588 4.923647741748382

MSEs: 57.38168678632161 48.79685160338582

**MSE still high (possibly due to this being a large data set**

## Save processed data

In [ ]:
#save vacancy rate data for modeling - remember to use random state=42!
#df.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/processed/VacancyRate_Zillow_2014_2018', index=False)
#df_2019_2020.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/processed/VacancyRate_Zillow_2019_2020', index=False)

In [ ]:
#save the scaled training and test splits

#X_tr_scaled.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/processed/X_tr_scaled', index=False)
#X_te_scaled

### Summary
This summary should provide a quick overview for someone wanting to know quickly why the given model was chosen for the next part of the business problem to help guide important business decisions.

- dropped margin of error of vacancy rate as this is correlated with vacancy rate and we do not have variables in 2019-2020 thus would not serve us well in a predictive model
- inspected data and created dummy variables for categorical variables (ie. metro, state, city, county name)
- split into testing and training datasets 
- estabilshed baseline measurement comparisons with dummy regressors
- attempted to fit the training data on a linear regression model...
**Note: THIS NOTEBOOK WAS NOT COMPLETED BECAUSE THE .FIT() FUNCTION TOOK FAR TOO LONG**
    - model fitting for Linear regression, on 5000 rows. Took 69.6165668964386 seconds with -7 R2 score
    - model fitting for Linear regression, on 10000 rows. Took 711.7782809734344 seconds with -6.32 R2 score
- did not complete notebook

### Reflection: 
**Review the following questions and apply them to your dataset**:

● Does my data set have any categorical data, such as Gender or day of the week? **Yes noted above in summary**

● Do my features have data values that range from 0 - 100 or 0-1 or both and more **yes vacancy rate is 0-100**